In [ ]:
# Step 0: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Step 1: Install dependencies and import libraries
!pip install -q transformers
!pip install -q torchinfo
!pip install -q datasets
!pip install -q evaluate
!pip install -q optuna
!pip install -q wandb

import wandb
wandb.login(key="Put wandb API key here")

from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from datasets import load_dataset
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import optuna
import os


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cu

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: peng_zhao (peng_zhao-university-of-california-berkeley) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
# Step 2: Load and preprocess the dataset (CDs_and_Vinyl)
dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_review_CDs_and_Vinyl", trust_remote_code=True)

# Shuffle the dataset and filter out rating == 3
shuffled_dataset = dataset["full"].shuffle(seed=42)
subset_dataset = shuffled_dataset.filter(lambda x: x["rating"] != 3)

# Initialize the DeBERTa tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-small")

# Define the tokenization and formatting function
def tokenize_and_format(examples):
    outputs = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=256)
    outputs["labels"] = [1 if rating > 3 else 0 for rating in examples["rating"]]
    return outputs

# Tokenize and save checkpoint
tokenized_dataset = subset_dataset.map(tokenize_and_format, batched=True)
tokenized_dataset.save_to_disk("/content/drive/MyDrive/FP/Checkpoints/tokenized_CDs_checkpoint")



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/30.3k [00:00<?, ?B/s]

Amazon-Reviews-2023.py:   0%|          | 0.00/39.6k [00:00<?, ?B/s]

CDs_and_Vinyl.jsonl:   0%|          | 0.00/3.29G [00:00<?, ?B/s]

Generating full split: 0 examples [00:00, ? examples/s]

...

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Map:   0%|          | 0/4543582 [00:00<?, ? examples/s]

Saving the dataset (0/20 shards):   0%|          | 0/4543582 [00:00<?, ? examples/s]

In [ ]:
# Step 2': Load from checkpoint if needed
from datasets import load_from_disk
tokenized_dataset = load_from_disk("/content/drive/MyDrive/FP/Checkpoints/tokenized_CDs_checkpoint")
tokenized_datasets = tokenized_dataset.train_test_split(test_size=0.2, seed=42)
tokenized_datasets.set_format("torch")


Loading dataset from disk:   0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
# Step 3: Define model, metrics, and hyperparameter search space
def model_init():
    return AutoModelForSequenceClassification.from_pretrained("microsoft/deberta-v3-small", num_labels=2)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

def hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-4, log=True),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [8, 16])
    }

# Set training arguments for hyperparameter tuning
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=256,
    per_device_eval_batch_size=128,
    gradient_accumulation_steps=1,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    fp16=True,
    dataloader_num_workers=8,
    warmup_steps=500,
    report_to=["wandb"],
    run_name="deberta_amazon_reviews_tune",
    logging_steps=50,
    logging_first_step=True
)

# Print total samples
total_train = len(tokenized_datasets["train"])
total_eval = len(tokenized_datasets["test"])
print("Total training samples:", total_train)
print("Total evaluation samples:", total_eval)

# Use partial subset for hyperparameter search
train_subset = tokenized_datasets["train"].shuffle(seed=42).select(range(int(0.002 * total_train)))
eval_subset = tokenized_datasets["test"].shuffle(seed=42).select(range(int(0.002 * total_eval)))

# Initialize the Trainer
trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=train_subset,
    eval_dataset=eval_subset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

# Run Optuna hyperparameter search
best_run = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    n_trials=10,
    hp_space=hp_space,
)

print("Best run hyperparameters:", best_run.hyperparameters)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Total training samples: 3634865
Total evaluation samples: 908717


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/286M [00:00<?, ?B/s]

...

[I 2025-04-03 04:58:52,493] Trial 8 pruned. 
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


eval/accuracy,▁█
eval/f1,▁█
eval/loss,█▁
eval/precision,▁█
eval/recall,▁█
eval/runtime,▁█
eval/samples_per_second,█▁
eval/steps_per_second,█▁
train/epoch,▁▁▂▂▃▃▃▄▄▄▄▅▅▆▆▆▇▇███
train/global_step,▁▁▂▂▃▃▃▄▄▄▄▅▅▆▆▆▇▇███
train/grad_norm,▆▆▆▆▅▂▂▂▃▂▃▅▁▆▅█▁▇▄


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.139200,0.104566,0.959824,0.959094,0.958515,0.959824
2,0.123600,0.109165,0.959824,0.962831,0.968914,0.959824


[I 2025-04-03 05:01:32,592] Trial 9 pruned. 


Best run hyperparameters: {'learning_rate': 1.2026872819004038e-05, 'per_device_train_batch_size': 8}


In [ ]:
# Step 4: Final training with best hyperparameters
updated_training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=best_run.hyperparameters["learning_rate"],
    per_device_train_batch_size=best_run.hyperparameters["per_device_train_batch_size"],
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    report_to=["wandb"],
    run_name="deberta_amazon_reviews_final",
    logging_steps=50
)

# Reload subset for final training
train_subset_final = tokenized_datasets["train"].shuffle(seed=42).select(range(int(0.002 * total_train)))
eval_subset_final = tokenized_datasets["test"].shuffle(seed=42).select(range(int(0.002 * total_eval)))

# Train final model
final_trainer = Trainer(
    model_init=model_init,
    args=updated_training_args,
    train_dataset=train_subset_final,
    eval_dataset=eval_subset_final,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

train_result = final_trainer.train()


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.117000,0.134048,0.964227,0.963113,0.962454,0.964227
2,0.069600,0.140895,0.964777,0.966471,0.969401,0.964777
3,0.054900,0.138558,0.968630,0.968569,0.968510,0.968630


In [ ]:
# Save final model
final_trainer.save_model("/content/drive/MyDrive/FP/Checkpoints/final_checkpoint_CDs_deberta")